In [2]:
from bokeh.plotting import figure, output_file, output_notebook, show, ColumnDataSource
from bokeh.sampledata import us_states
from bokeh.models import HoverTool
from bokeh.plotting import *
import pandas as pd
import dask.dataframe as dd

us_states = us_states.data.copy()

del us_states["HI"]
del us_states["AK"]

# separate latitude and longitude points for the borders of the states.
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]

stations = dd.read_csv('_stations.csv')
turbines = dd.read_csv('_turbines.csv')


# filter latitudes and longitudes to be in the East Coast of US as downsampling method
sta_query_frags = [
    'long > -120 and long < -50',
    'lat > 25 and lat < 50'
]

stations = stations.query(' and '.join(sta_query_frags))

tur_query_frags = [
    'xlong > -120 and xlong < -50',
    'ylat > 25 and ylat < 50'
]

turbines = turbines.query(' and '.join(tur_query_frags))
# max_wind and mean_wind
stations_source = ColumnDataSource(data=dict(
    x=[i for i in stations['long']],
    y=[i for i in stations['lat']],
    mean_2019_wind_speeds=[i for i in stations['mean_wind']], 
    maximum_2019_wind_speed=[i for i in stations['max_wind']]
))


# case_id  t_manu     t_model   t_cap  t_hh   t_rd     t_rsa  t_ttlh
turbines_source = ColumnDataSource(data=dict(
    x=[i for i in turbines['xlong']],
    y=[i for i in turbines['ylat']],
    turbine_capacity=[i for i in turbines['t_cap']],
    turbine_manufacturer=[i for i in turbines['t_manu']],
    turbine_model=[i for i in turbines['t_model']],
    turbine_height_metres=[i for i in turbines['t_hh']],
    turbine_rotor_diameter_metres=[i for i in turbines['t_rd']]
))

TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
    ("turbine_manufacturer", "@turbine_manufacturer"),
    ("turbine_model", "@turbine_model"),
    ("turbine_capacity_kilowatts", "@turbine_capacity"),
    ("turbine_height_metres", "@turbine_height_metres"),
    ("turbine_rotor_diameter_metres","@turbine_rotor_diameter_metres")
]



p = figure(plot_width=1200, plot_height=800,
           title="Weather Stations and Wind Turbines in US")

p.patches(state_xs, state_ys, fill_alpha=0.0, line_color="#884444", line_width=1.5)


# Weather Stations
circles = p.circle('x', 'y', size=4, color="orange", legend_label="Weather Stations", source=stations_source)

# Wind Turbines
squares = p.square('x', 'y', size=3.5,color="navy", legend_label="Wind Turbines", source=turbines_source)



hover_turbines = HoverTool(renderers=[squares]) # only for circles


hover_turbines.tooltips = [
    ("(x,y)", "($x, $y)"),
    ("turbine_manufacturer", "@turbine_manufacturer"),
    ("turbine_model", "@turbine_model"),
    ("turbine_capacity_kilowatts", "@turbine_capacity"),
    ("turbine_height_metres", "@turbine_height_metres"),
    ("turbine_rotor_diameter_metres","@turbine_rotor_diameter_metres")
]
p.add_tools(hover_turbines)


hover_stations = HoverTool(renderers=[circles])
hover_stations.tooltips = [
    ("(x,y)", "($x, $y)"),
    ("mean_2019_wind_speeds(knots per hour)", "@mean_2019_wind_speeds"),
    ("maximum_2019_wind_speed(knots per hour)", "@maximum_2019_wind_speed")
]
p.add_tools(hover_stations)

# output to static HTML file
#output_file("us_wind_turbines_demo.html")
output_notebook()
# show results
show(p)

Loading BokehJS ...